# Preparação do ambiente

## Bibliotecas

In [1]:
import json
import pandas as pd
import re

from pprint import pprint

## Funções auxiliares

In [2]:
NULL_STRING = '#NULLO#'

In [3]:
def flatten_matrix(matrix):
    return [item for row in matrix for item in row]

In [4]:
def extract_amazon_images(row):
    imagens = []
    imagens.append(row['Imagem'].split()[0])
    imagens.extend(row['Imagens'].split())

    imagens = [imagem for imagem in imagens if imagem.startswith('http')]
    if len(imagens) >= 3:
        imagens = imagens[:3]
    elif len(imagens) == 2:
        imagens.extend([''])
    elif len(imagens) == 1:
        imagens.extend(['',''])
    else:
        imagens.extend(['','',''])

    return pd.Series(imagens)

def extract_amazon_desc(row):
    desc = '\n'.join(row)
    desc = desc.replace(NULL_STRING,' ')
    desc = re.sub('\s+', ' ', desc)
    return desc

In [5]:
def extract_magalu_images(row):
    imagens = []
    imagens.append(row['Imagem'].split()[0])
    imagens.extend(row['Imagens'].split(','))

    imagens = [imagem for imagem in imagens if imagem.startswith('http')]
    if len(imagens) >= 3:
        imagens = imagens[:3]
    elif len(imagens) == 2:
        imagens.extend([''])
    elif len(imagens) == 1:
        imagens.extend(['',''])
    else:
        imagens.extend(['','',''])

    return pd.Series(imagens)

# Carga e prepação dos dados

## Amazon

In [6]:
file_amazon_reindexed = '../datasets/amazon_reindexed.parquet'
df_amazon = pd.read_parquet(file_amazon_reindexed)

columns_to_keep = ['Link', 'Nome', 'Descrição', 'Descrição_Principal', 'Imagem', 'Imagens' ]
df_amazon = df_amazon[columns_to_keep]

df_amazon['desc'] = df_amazon[['Descrição', 'Descrição_Principal']].apply(lambda row: extract_amazon_desc(row),axis=1)
# df_amazon['img_list'] = df_amazon[['Imagem', 'Imagens']].apply(lambda row: extract_amazon_images(row),axis=1)
df_amazon[['img0','img1','img2']] = df_amazon[['Imagem', 'Imagens']].apply(lambda row: extract_amazon_images(row),axis=1)

columns_to_keep = ['Link', 'Nome', 'desc', 'img0','img1','img2']
df_amazon = df_amazon[columns_to_keep]
df_amazon.columns = ['link', 'nome', 'desc', 'img0', 'img1', 'img2']
df_amazon.head()

,link,nome,desc,img0,img1,img2
0,https://www.amazon.com.br/Especiais-Cinematic-...,Filtro de Lente de Efeitos Especiais Blue Stre...,Especificações: Modelo aplicável: este filtro ...,https://m.media-amazon.com/images/I/61ZdffRDj2...,https://m.media-amazon.com/images/I/51DeP4IE2F...,https://m.media-amazon.com/images/I/41w7kz5r4P...
1,https://www.amazon.com.br/Motores-Escova-Conju...,"Motores Sem Escova, 4 Conjuntos de Motores Sem...",1. Modelo de uso: Este motor sem escova de dro...,https://m.media-amazon.com/images/I/51aZmDncic...,https://m.media-amazon.com/images/I/31mER3G37I...,https://m.media-amazon.com/images/I/31jk4g-fUt...
2,https://www.amazon.com.br/Armazenamento-Protet...,"Bolsa de Armazenamento Protetora para Drone, C...",1. Excelente proteção: o case rígido do drone ...,https://m.media-amazon.com/images/I/612uWGXrp7...,https://m.media-amazon.com/images/I/41JfSjIxxR...,https://m.media-amazon.com/images/I/41CVth1dqO...
3,https://www.amazon.com.br/EQUSS-V-strom-Motoci...,EQUSS Para DUCATI DL650 DL1000 V-strom DL 650 ...,1. Material: liga de alumínio [Super Grip] - D...,https://m.media-amazon.com/images/I/51-m8iNiuL...,https://m.media-amazon.com/images/I/31VLMQCsxU...,https://m.media-amazon.com/images/I/31WjqM+BqB...
4,https://www.amazon.com.br/remote-control-brush...,"remote control car brushless motor servo kit,r...",,https://m.media-amazon.com/images/I/61Hzfl2r75...,https://m.media-amazon.com/images/I/41P1kyyx1l...,https://m.media-amazon.com/images/I/41I9plec-T...


## Magalu

In [7]:
file_magalu_reindexed = '../datasets/magalu_redindexed.parquet.gzip'
df_magalu = pd.read_parquet(file_magalu_reindexed)

columns_to_keep = ['Link', 'Nome', 'Descrição', 'Imagem', 'Imagens' ]
df_magalu = df_magalu[columns_to_keep]

df_magalu[['img0','img1','img2']] = df_magalu[['Imagem', 'Imagens']].apply(lambda row: extract_magalu_images(row),axis=1)
columns_to_keep = ['Link', 'Nome', 'Descrição', 'img0','img1','img2']
df_magalu = df_magalu[columns_to_keep]
df_magalu.columns = ['link', 'nome', 'desc', 'img0', 'img1', 'img2']

df_magalu.sample(5)

,link,nome,desc,img0,img1,img2
11315,https://validate.perfdrive.com/ca4df1c7abf7ea2...,Pedimos desculpas pela inconveniência...,#NULLO#,https://a-static.mlcdn.com.br/280x210/10-lugar...,,
42936,https://www.magazineluiza.com.br/jammer-bermud...,Jammer Bermuda de Natação Arena Powerskin ST 2...,Jammer Bermuda de Natação Arena Powerskin ST 2...,https://a-static.mlcdn.com.br/280x210/jammer-b...,https://a-static.mlcdn.com.br/90x90/jammer-ber...,
7159,https://validate.perfdrive.com/ca4df1c7abf7ea2...,Pedimos desculpas pela inconveniência...,#NULLO#,https://a-static.mlcdn.com.br/280x210/aquecedo...,,
5198,https://validate.perfdrive.com/ca4df1c7abf7ea2...,Pedimos desculpas pela inconveniência...,#NULLO#,https://a-static.mlcdn.com.br/280x210/1-unidad...,,
549,https://validate.perfdrive.com/ca4df1c7abf7ea2...,Pedimos desculpas pela inconveniência...,#NULLO#,https://a-static.mlcdn.com.br/280x210/fone-de-...,,


## Mercado Livre

In [8]:
file_ml_reindexed = '../datasets/ml_redindexed.parquet.gzip'
df_ml = pd.read_parquet(file_ml_reindexed)

columns_to_keep = ['Link', 'Nome', 'Descrição', 'Imagem', 'Imagens' ]
df_ml = df_ml[columns_to_keep]

df_ml[['img0','img1','img2']] = df_ml[['Imagem', 'Imagens']].apply(lambda row: extract_magalu_images(row),axis=1)
columns_to_keep = ['Link', 'Nome', 'Descrição', 'img0','img1','img2']
df_ml = df_ml[columns_to_keep]
df_ml.columns = ['link', 'nome', 'desc', 'img0', 'img1', 'img2']

df_ml.sample(5)

,link,nome,desc,img0,img1,img2
54615,https://produto.mercadolivre.com.br/MLB-405145...,Campainha Inteligente Porteiro Eletrônico Film...,#NULLO#,https://http2.mlstatic.com/D_NQ_NP_989820-MLB7...,,
73681,https://www.mercadolivre.com.br/smart-tv-hq-60...,"Smart TV HQ 60"" UHD 4K, HDR Android 11, Design...",Com TV HQSTV60NK você poderá desfrutar de seu ...,https://http2.mlstatic.com/D_NQ_NP_642902-MLU7...,,
45984,https://produto.mercadolivre.com.br/MLB-369650...,Smart Tv Box Aquario - Homologado Anatel - Net...,OBS: O aparelho Tv Box serve somente para o us...,https://http2.mlstatic.com/D_NQ_NP_731427-MLB5...,,
47080,https://produto.mercadolivre.com.br/MLB-374296...,Sistema Sem Fio Do Receptor Do Transmissor Da ...,Característica: 1. O produto inclui um transmi...,https://http2.mlstatic.com/D_NQ_NP_848178-CBT7...,https://http2.mlstatic.com/D_NQ_NP_853757-CBT7...,https://http2.mlstatic.com/D_NQ_NP_749559-CBT7...
16174,https://produto.mercadolivre.com.br/MLB-280370...,Receptor Digital Full Hd Satmax 5 Banda C Ou K...,Não deixe pra ultima hora! Atualize já o seu k...,https://http2.mlstatic.com/D_NQ_NP_621995-MLB5...,,


# União dos dados em único dataframe

In [9]:
df_anuncios = pd.concat([df_amazon, df_magalu, df_ml])
df_anuncios = df_anuncios.reset_index(drop=True)
df_anuncios.head(5)

,link,nome,desc,img0,img1,img2
0,https://www.amazon.com.br/Especiais-Cinematic-...,Filtro de Lente de Efeitos Especiais Blue Stre...,Especificações: Modelo aplicável: este filtro ...,https://m.media-amazon.com/images/I/61ZdffRDj2...,https://m.media-amazon.com/images/I/51DeP4IE2F...,https://m.media-amazon.com/images/I/41w7kz5r4P...
1,https://www.amazon.com.br/Motores-Escova-Conju...,"Motores Sem Escova, 4 Conjuntos de Motores Sem...",1. Modelo de uso: Este motor sem escova de dro...,https://m.media-amazon.com/images/I/51aZmDncic...,https://m.media-amazon.com/images/I/31mER3G37I...,https://m.media-amazon.com/images/I/31jk4g-fUt...
2,https://www.amazon.com.br/Armazenamento-Protet...,"Bolsa de Armazenamento Protetora para Drone, C...",1. Excelente proteção: o case rígido do drone ...,https://m.media-amazon.com/images/I/612uWGXrp7...,https://m.media-amazon.com/images/I/41JfSjIxxR...,https://m.media-amazon.com/images/I/41CVth1dqO...
3,https://www.amazon.com.br/EQUSS-V-strom-Motoci...,EQUSS Para DUCATI DL650 DL1000 V-strom DL 650 ...,1. Material: liga de alumínio [Super Grip] - D...,https://m.media-amazon.com/images/I/51-m8iNiuL...,https://m.media-amazon.com/images/I/31VLMQCsxU...,https://m.media-amazon.com/images/I/31WjqM+BqB...
4,https://www.amazon.com.br/remote-control-brush...,"remote control car brushless motor servo kit,r...",,https://m.media-amazon.com/images/I/61Hzfl2r75...,https://m.media-amazon.com/images/I/41P1kyyx1l...,https://m.media-amazon.com/images/I/41I9plec-T...


In [11]:
df_anuncios.to_parquet('../datasets/anuncios.parquet')

# Extratir amostra

In [19]:
n_samples = 20000
df_sample_to_label = df_anuncios.iloc[:,1:].sample(n_samples,random_state=111)
df_sample_to_label.head()

,nome,desc,img0,img1,img2
246822,"Adaptador Bluetooth Tp Link, 5.0, Usb Nano, Ub500",O ub500 equipa o seu pc com a tecnologia bluet...,https://http2.mlstatic.com/D_NQ_NP_657738-MLB5...,https://http2.mlstatic.com/D_NQ_NP_619224-MLB5...,
77666,INOOMP 1Pc Gerador De Mão Portátil Carregador ...,,,,
271778,Transmissor Receptor De Guitarra Sem Fio Cabo ...,Transmissores e receptores sem fio podem ser u...,https://http2.mlstatic.com/D_NQ_NP_645594-CBT7...,,
264205,Camera De Segurança Q6 Solar Wi-fi A Prova D'á...,"Lembrete gentil:\n1. Olá, por favor tirar toda...",https://http2.mlstatic.com/D_NQ_NP_952743-MLB7...,https://http2.mlstatic.com/D_NQ_NP_731137-MLB7...,https://http2.mlstatic.com/D_NQ_NP_790464-MLB7...
281503,Fone de ouvido Bluetooth in-ear gamer sem fio ...,FONE BLUETOOTH XIAOMI REDMI AIRDOTS 2 DEVIDAME...,https://http2.mlstatic.com/D_NQ_NP_939695-MLA4...,,


In [20]:
# Lista de arquivos para baixar
images = []
images.extend(df_sample_to_label['img0'].to_list())
images.extend(df_sample_to_label['img1'].to_list())
images.extend(df_sample_to_label['img2'].to_list())
images = [f'wget {image}\n' for image in images if len(image)>0]
with open('images_to_download.sh','w') as f:
    f.writelines(images)

In [21]:
for col in ['img0', 'img1', 'img2']:
    df_sample_to_label[col] = df_sample_to_label[col].apply(lambda x: f'/data/local-files/?d=images/{x.split("/")[-1]}' if len(x) > 0 else '/data/local-files/?d=images/none.jpg')
df_sample_to_label.head()

,nome,desc,img0,img1,img2
246822,"Adaptador Bluetooth Tp Link, 5.0, Usb Nano, Ub500",O ub500 equipa o seu pc com a tecnologia bluet...,/data/local-files/?d=images/D_NQ_NP_657738-MLB...,/data/local-files/?d=images/D_NQ_NP_619224-MLB...,/data/local-files/?d=images/none.jpg
77666,INOOMP 1Pc Gerador De Mão Portátil Carregador ...,,/data/local-files/?d=images/none.jpg,/data/local-files/?d=images/none.jpg,/data/local-files/?d=images/none.jpg
271778,Transmissor Receptor De Guitarra Sem Fio Cabo ...,Transmissores e receptores sem fio podem ser u...,/data/local-files/?d=images/D_NQ_NP_645594-CBT...,/data/local-files/?d=images/none.jpg,/data/local-files/?d=images/none.jpg
264205,Camera De Segurança Q6 Solar Wi-fi A Prova D'á...,"Lembrete gentil:\n1. Olá, por favor tirar toda...",/data/local-files/?d=images/D_NQ_NP_952743-MLB...,/data/local-files/?d=images/D_NQ_NP_731137-MLB...,/data/local-files/?d=images/D_NQ_NP_790464-MLB...
281503,Fone de ouvido Bluetooth in-ear gamer sem fio ...,FONE BLUETOOTH XIAOMI REDMI AIRDOTS 2 DEVIDAME...,/data/local-files/?d=images/D_NQ_NP_939695-MLA...,/data/local-files/?d=images/none.jpg,/data/local-files/?d=images/none.jpg


In [26]:
if 'id' not in df_sample_to_label.columns:
    print('Create id column')
    df_sample_to_label = df_sample_to_label.reset_index()
    new_columns = ['id', 'nome', 'desc', 'img0', 'img1', 'img2']
    df_sample_to_label.columns = new_columns
df_sample_to_label.head()

,id,nome,desc,img0,img1,img2
0,246822,"Adaptador Bluetooth Tp Link, 5.0, Usb Nano, Ub500",O ub500 equipa o seu pc com a tecnologia bluet...,/data/local-files/?d=images/D_NQ_NP_657738-MLB...,/data/local-files/?d=images/D_NQ_NP_619224-MLB...,/data/local-files/?d=images/none.jpg
1,77666,INOOMP 1Pc Gerador De Mão Portátil Carregador ...,,/data/local-files/?d=images/none.jpg,/data/local-files/?d=images/none.jpg,/data/local-files/?d=images/none.jpg
2,271778,Transmissor Receptor De Guitarra Sem Fio Cabo ...,Transmissores e receptores sem fio podem ser u...,/data/local-files/?d=images/D_NQ_NP_645594-CBT...,/data/local-files/?d=images/none.jpg,/data/local-files/?d=images/none.jpg
3,264205,Camera De Segurança Q6 Solar Wi-fi A Prova D'á...,"Lembrete gentil:\n1. Olá, por favor tirar toda...",/data/local-files/?d=images/D_NQ_NP_952743-MLB...,/data/local-files/?d=images/D_NQ_NP_731137-MLB...,/data/local-files/?d=images/D_NQ_NP_790464-MLB...
4,281503,Fone de ouvido Bluetooth in-ear gamer sem fio ...,FONE BLUETOOTH XIAOMI REDMI AIRDOTS 2 DEVIDAME...,/data/local-files/?d=images/D_NQ_NP_939695-MLA...,/data/local-files/?d=images/none.jpg,/data/local-files/?d=images/none.jpg


In [33]:
samples = df_sample_to_label.to_dict('records')
samples[0]
with open('../datasets/samples_to_label.json','w') as f:
    json.dump(samples,f)


pprint(samples[0])

{'desc': 'O ub500 equipa o seu pc com a tecnologia bluetooth 5.0 mais avançada '
         'que possui atualizações gerais em comparação com as versões mais '
         'antigas. Ele executa velocidades de conexão mais rápidas e maior '
         'alcance do que a tecnologia bluetooth 4.0, garantindo uma conexão '
         'sem fio forte e estável entre seu pc e dispositivos bluetooth. Os '
         'usuários também se beneficiam da economia de energia e maior '
         'capacidade de transmissão. Bluetooth 5.0 – aplica a mais recente '
         'tecnologia bluetooth 5.0, compatível com bluetooth '
         'v4.0/3.0/2.1/2.0/1.1 conectividade wireless – fornece comunicação '
         'estável e conveniente entre dispositivos bluetooth e seu pc ou '
         'notebook tamanho nano - ultra-pequeno para portabilidade conveniente '
         'com alto desempenho confiável compatível com sistema operacional – '
         'windows 11/10/8.1/7\n'
         'CARACTERÍSTICAS:\n'
         '- Marca:Tp